In [1]:
from tika import parser

# tika.initVM()

filepath = 'upload/Deep-learning-in-insurance--Accuracy-and-model-int_2023_Expert-Systems-with-.pdf'  # Replace with the path of your file

parsed_document = parser.from_file(filepath)

# print(parsed_document)

print(type(parsed_document))

print(parsed_document['metadata'])  
  
# <class 'dict'> 
print(type(parsed_document['metadata']))

2024-09-09 21:29:58,901 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2024-09-09 21:30:50,727 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2024-09-09 21:30:51,835 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


<class 'dict'>
{'pdf:PDFVersion': '1.7', 'xmp:CreatorTool': 'Elsevier', 'pdf:docinfo:title': 'Deep learning in insurance: Accuracy and model interpretability using TabNet', 'pdf:hasXFA': 'false', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dc:format': 'application/pdf; version=1.7', 'pdf:docinfo:custom:robots': 'noindex', 'pdf:docinfo:creator_tool': 'Elsevier', 'access_permission:fill_in_form': 'true', 'pdf:docinfo:custom:CreationDate--Text': '7th February 2023', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'dc:title': 'Deep learning in insurance: Accuracy and model interpretability using TabNet', 'pdf:containsNonEmbeddedFont': 'false', 'xmp:CreateDate': '2023-02-04T09:40:15Z', 'pdf:docinfo:custom:CrossMarkDomains[1]': 'elsevier.com', 'robots': 'noindex', 'pdf:docinfo:subject': 'Expe